In [4]:
import os
import numpy as np
import pandas as pd
from sklearn import preprocessing
from torch.utils.data import Dataset, DataLoader
from sklearn.preprocessing import StandardScaler
import torch
from torch import nn
from torch.autograd import Variable
import torch.nn.functional as F
from sklearn.model_selection import train_test_split

In [9]:
class PatientVectorDataset(Dataset):
    """Patient vector dataset."""

    def __init__(self, csv_file):
        """
        Args:
            csv_file (string): Path to the csv file with vectors.
        """
        self.patientvecs = pd.read_csv(csv_file)

    def __len__(self):
        return len(self.patientvecs)

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()

        patient = self.patientvecs.iloc[idx, 1:]
        patient = np.array([patient])
        #patient = landmarks.astype('float').reshape(-1, 2)

        return patient

In [10]:
patientvecs_dataset = PatientVectorDataset(csv_file='/Users/nicenoize/Documents/DATEXIS/DeepPatient/test_multi_hot.csv')
dataloader = DataLoader(patientvecs_dataset, batch_size=4, shuffle=True, num_workers=4)
dataset = pd.read_csv('/Users/nicenoize/Documents/DATEXIS/DeepPatient/test_multi_hot.csv')
# X = Features
# y = diagnoses
X = dataset.drop(dataset.columns[0], axis=1)
y = dataset[dataset.columns[0]]
x_train,x_test,y_train,y_test=train_test_split(X,y,test_size=0.2)

In [11]:
class CDAutoEncoder(nn.Module):
    """
    Convolutional denoising autoencoder layer for stacked autoencoders.
    This module is automatically trained when in model.training is True.

    Args:
        input_size: The number of features in the input
        output_size: The number of features to output
        stride: Stride of the convolutional layers.
    """
    def __init__(self, input_size, output_size, stride):
        super(CDAutoEncoder, self).__init__()

        self.forward_pass = nn.Sequential(
            nn.Conv2d(input_size, output_size, kernel_size=2, stride=stride, padding=0),
            nn.ReLU(),
        )
        self.backward_pass = nn.Sequential(
            nn.ConvTranspose2d(output_size, input_size, kernel_size=2, stride=2, padding=0), 
            nn.ReLU(),
        )

        self.criterion = nn.MSELoss()
        self.optimizer = torch.optim.SGD(self.parameters(), lr=0.1)

    def forward(self, x):
        # Train each autoencoder individually
        x = x.detach()
        # Add noise, but use the original lossless input as the target.
        x_noisy = x * (Variable(x.data.new(x.size()).normal_(0, 0.1)) > -.1).type_as(x)
        y = self.forward_pass(x_noisy)

        if self.training:
            x_reconstruct = self.backward_pass(y)
            #loss = self.criterion(x_reconstruct, Variable(x.data, requires_grad=False))
            loss = nn.CrossEntropyLoss()
            self.optimizer.zero_grad()
            loss.backward()
            self.optimizer.step()
            
        return y.detach()

    def reconstruct(self, x):
        return self.backward_pass(x)


class StackedAutoEncoder(nn.Module):
    """
    A stacked autoencoder made from the convolutional denoising autoencoders above.
    Each autoencoder is trained independently and at the same time.
    """

    def __init__(self):
        super(StackedAutoEncoder, self).__init__()

        #self.ae1 = CDAutoEncoder(500, 500, 500)
        #self.ae2 = CDAutoEncoder(500, 500, 500)
        #self.ae3 = CDAutoEncoder(500, 500, 500)
        #input_size, output_size, kernel_size=2, stride=stride, padding=0)
        self.ae1 = CDAutoEncoder(80, 4, 80)
        self.ae2 = CDAutoEncoder(80, 4, 80)
        self.ae3 = CDAutoEncoder(80, 4, 80)

        
    def forward(self, x):
        a1 = self.ae1(x)
        a2 = self.ae2(a1)
        a3 = self.ae3(a2)

        if self.training:
            return a3

        else:
            return a3, self.reconstruct(a3)

    def reconstruct(self, x):
            a2_reconstruct = self.ae3.reconstruct(x)
            a1_reconstruct = self.ae2.reconstruct(a2_reconstruct)
            x_reconstruct = self.ae1.reconstruct(a1_reconstruct)
            return x_reconstruct


In [13]:
import os
import time

import torch
import torchvision
from torch import nn
from torch.autograd import Variable
from torch.utils.data import DataLoader
from torchvision import transforms
from torchvision.datasets import MNIST, CIFAR10
from torchvision.utils import save_image

num_epochs = 1000
batch_size = 128

model = StackedAutoEncoder() #.cuda()

for epoch in range(num_epochs):
    if epoch % 10 == 0:
        # Test the quality of our features with a randomly initialzed linear classifier.
        classifier = nn.Linear(512 * 16, 10) #.cuda()
        criterion = nn.CrossEntropyLoss()
        optimizer = torch.optim.Adam(classifier.parameters(), lr=0.001)

    model.train()
    total_time = time.time()
    correct = 0
    for i, patient in enumerate(dataloader):
        patient = Variable(patient)#.cuda()
        features = model(patient).detach()
        prediction = classifier(features.view(features.size(0), -1))
        loss = criterion(prediction, patient)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        pred = prediction.data.max(1, keepdim=True)[1]
        correct += pred.eq(target.data.view_as(pred)).cpu().sum()

    total_time = time.time() - total_time

    model.eval()
    patient, _ = data
    patient = Variable(patient)#.cuda()
    features, x_reconstructed = model(patient)
    reconstruction_loss = torch.mean((x_reconstructed.data - patient.data)**2)

    if epoch % 10 == 0:
        print("Saving epoch {}".format(epoch))
        #save_image(orig, './imgs/orig_{}.png'.format(epoch))
        #pic = to_img(x_reconstructed.cpu().data)
        #save_image(pic, './imgs/reconstruction_{}.png'.format(epoch))

    print("Epoch {} complete\tTime: {:.4f}s\t\tLoss: {:.4f}".format(epoch, total_time, reconstruction_loss))
    print("Feature Statistics\tMean: {:.4f}\t\tMax: {:.4f}\t\tSparsity: {:.4f}%".format(
        torch.mean(features.data), torch.max(features.data), torch.sum(features.data == 0.0)*100 / features.data.numel())
    )
    print("Linear classifier performance: {}/{} = {:.2f}%".format(correct, len(dataloader)*batch_size, 100*float(correct) / (len(dataloader)*batch_size)))
    print("="*80)

torch.save(model.state_dict(), './CDAE.pth')


RuntimeError: Expected 4-dimensional input for 4-dimensional weight [4, 80, 2, 2], but got 3-dimensional input of size [4, 1, 80] instead